In [1]:
import pennylane as qml
import pennylane.numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Module 1

In [2]:
def coefficients_to_values(coefficients):
    """Returns the value representation of a polynomial
    
    Args:
        coefficients (array[complex]): a 1-D array of complex 
            coefficients of a polynomial with 
            index i representing the i-th degree coefficient

    Returns: 
        array[complex]: the value representation of the 
            polynomial 
    """
    ##################
    # YOUR CODE HERE #
    ################## 
    return np.fft.fft(coefficients)

A = [4, 3, 2, 1]
print(coefficients_to_values(A))

[10.+0.j  2.-2.j  2.+0.j  2.+2.j]


In [3]:
def coefficients_to_values(coefficients):
    """Returns the value representation of a polynomial
    
    Args:
        coefficients (array[complex]): a 1-D array of complex 
            coefficients of a polynomial with 
            index i representing the i-th degree coefficient

    Returns: 
        array[complex]: the value representation of the 
            polynomial 
    """
    ##################
    # YOUR CODE HERE #
    ################## 
    return np.fft.fft(coefficients)

A = [4, 3, 2, 1]
print(coefficients_to_values(A))


[10.+0.j  2.-2.j  2.+0.j  2.+2.j]


In [4]:
def values_to_coefficients(values):
   #Compute the action U_IDFT on \tilde{A}(x) using the numpy inverse FFT function
    return np.fft.ifft(values)

A = [10.+0.j,  2.-2.j,  2.+0.j,  2.+2.j]
print(values_to_coefficients(A))

[4.+0.j 3.+0.j 2.+0.j 1.+0.j]


In [5]:
def nearest_power_of_2(x):
    return 2 ** int(np.ceil(np.log2(x)))

In [6]:
def fft_multiplication(poly_a, poly_b):
    # Calculate the number of values required
    x = len(poly_a) + len(poly_b) -1

    # Figure out the nearest power of 2
    m = nearest_power_of_2(x)

    # Pad zeros to the polynomial
    poly_a = np.pad(poly_a, pad_width = (0, m - len(poly_a)), mode = 'constant')
    poly_b = np.pad(poly_b, pad_width = (0, m - len(poly_b)), mode = 'constant')

    # Compute the value representation of each polynomial using DFT
    val_a = coefficients_to_values(poly_a)
    val_b = coefficients_to_values(poly_b)

    # Compute the dot product of value representation vectors
    val_c = np.multiply(val_a, val_b)
    
    # Convert back to coefficient representation using IDFT
    poly_c = values_to_coefficients(val_c)
    
    return poly_c

# Module 2

In [7]:
dev = qml.device("default.qubit", wires=1)

@qml.qnode(dev)
def one_qubit_QFT(basis_id):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=1)]
    qml.BasisStatePreparation(bits, wires=[0])

    qml.Hadamard(wires = 0)
    return qml.state()

In [8]:
n_bits = 2
dev = qml.device("default.qubit", wires=n_bits)

@qml.qnode(dev)
def two_qubit_QFT(basis_id):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_bits)]
    qml.BasisStatePreparation(bits, wires=[0, 1])
    
    #Define a function that raises omega to the nth power
    def omega(n):
        return np.exp(2 * np.pi * 1j/4 * n)
    
    #Create U_QFT
    U_QFT = (1/2) * np.array([[1,1,1,1], [1, omega(1), omega(2), omega(3)], 
    [1, omega(2), omega(4), omega(6)], [1, omega(3), omega(6), omega(9)]])
    
    #Apply it to the state
    qml.QubitUnitary(U_QFT, wires = [0,1])
    return qml.state()

In [9]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def decompose_two_qubit_QFT(basis_id):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=2)]
    qml.BasisStatePreparation(bits, wires=[0, 1])
    
    qml.Hadamard(wires = 0)
    qml.ctrl(qml.S(wires=0), control = 1)
    qml.Hadamard(wires = 1)
    qml.SWAP(wires = [0,1])
    return qml.state()

# Module 3

In [10]:
dev = qml.device("default.qubit", wires=3)

#This for some reason outputs "Incorrect: Your resulting output doesn't look quite right."
@qml.qnode(dev)
def three_qubit_QFT(basis_id):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=3)]
    qml.BasisStatePreparation(bits, wires=[0, 1, 2])
    
    #Note that R(2) = S gate and R(3) = T gate
    qml.Hadamard(wires=0)
    qml.ctrl(qml.S(wires=0), control=1)
    qml.ctrl(qml.T(wires=0), control=2)
    qml.Hadamard(wires=1)
    qml.ctrl(qml.S(wires=1), control=2)
    qml.Hadamard(wires=2)
    qml.SWAP(wires=[0,2])
    
    return qml.state()

In [11]:
dev = qml.device('default.qubit', wires=4)

            
def swap_bits(n_qubits):
    """A circuit that reverses the order of qubits, i.e.,
    performs a SWAP such that [q1, q2, ..., qn] -> [qn, ... q2, q1].
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    for wire in range(n_qubits//2):
        qml.SWAP([wire,n_qubits-wire-1])

@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    # qft_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()


In [12]:
dev = qml.device('default.qubit', wires=4)

def qft_rotations(n_qubits):
    """A circuit performs the QFT rotations on the specified qubits.
    
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
    """

    ##################
    # YOUR CODE HERE #
    ################## 
    for wire in range(n_qubits):
        qml.Hadamard(wire)
        for subwire in range(wire+1,n_qubits):
            qml.ControlledPhaseShift(np.pi/2**(subwire-wire),[subwire,wire])

@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    qft_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()


In [13]:
dev = qml.device('default.qubit', wires=4)

def qft_recursive_rotations(n_qubits, wire=0):
    """A circuit that performs the QFT rotations on the specified qubits
        recursively.
        
    Args:
        n_qubits (int): An integer value identifying the number of qubits.
        wire (int): An integer identifying the wire 
                    (or the qubit) to apply rotations on.
    """

    ##################
    # YOUR CODE HERE #
    ################## 
    if wire < n_qubits:
        qml.Hadamard(wire)
        for subwire in range(wire+1,n_qubits):
            qml.ControlledPhaseShift(np.pi/2**(subwire-wire),[subwire,wire])
        return qft_recursive_rotations(n_qubits,wire=wire+1)
        
    else:
        return

@qml.qnode(dev) 
def qft_node(basis_id, n_qubits):
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))
    qft_recursive_rotations(n_qubits)
    swap_bits(n_qubits)
    return qml.state()


In [14]:
dev = qml.device('default.qubit', wires=4)

@qml.qnode(dev)
def pennylane_qft(basis_id, n_qubits):
    """A that circuit performs the QFT using PennyLane's QFT template.
    
    Args:
        basis_id (int): An integer value identifying 
            the basis state to construct.
        n_qubits (int): An integer identifying the 
            number of qubits.
            
    Returns:
        array[complex]: The state after applying the QFT to the qubits.
    """
    # Prepare the basis state |basis_id>
    bits = [int(x) for x in np.binary_repr(basis_id, width=n_qubits)]
    qml.BasisStatePreparation(bits, wires=range(n_qubits))

    ##################
    # YOUR CODE HERE #
    ################## 
    qml.QFT(range(n_qubits))
    return qml.state()
